In [5]:
from pathlib import Path
import pandas as pd
import altair as alt
import seaborn as sns

In [20]:
Path.cwd()

PosixPath('/Users/ben/Developer/SpaceMissionDES/SpaceMissionDES/analysis')

#### Load the Data

In [21]:
data_dir = Path.cwd().parent.parent / "results" / "Case04-r04_10Tx-4Tk"

mc_results = pd.read_csv(data_dir / "mc.csv")
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,0,True,72.127086,2,6.000000,Tanker,Countdown
1,0,True,72.127086,3,37.762713,Tanker,Countdown
2,2,True,73.776831,2,6.000000,Tanker,Countdown
3,2,True,73.776831,3,37.762713,Tanker,Countdown
4,2,True,73.776831,4,45.110132,Tanker,Countdown


#### Summary Statistics

In [22]:
N = mc_results.replicant.max()
success_rate = sum(mc_results.groupby("replicant").first().outcome) / N
print(f"The success rate was: {success_rate} = {100*success_rate:.3f} %\n")

The success rate was: 0.8958958958958959 = 89.590 %



#### Mission Duration

In [23]:
df = (
    mc_results[mc_results.outcome == True]
    .groupby("replicant", as_index=False)
    .first()
    .loc[:, ["replicant", "duration"]]
)

alt.Chart(df).mark_bar().encode(
    # alt.X("duration:Q", bin=True),
    alt.X("duration:Q", bin=alt.Bin(extent=[50, 200], step=10)),
    y='count()',
).properties(
    width = 600
)

alt.Chart(...)

Compute the Emprical CDF

In [3]:
import numpy as np

# from: https://stackoverflow.com/questions/33345780/empirical-cdf-in-python-similiar-to-matlabs-one
def ecdf(sample):

    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob

def plot_ECDF(df):
    sample = df.duration.values
    qe, pe = ecdf(sample)
    data = pd.DataFrame({"duration": list(qe), "prob": list(pe)})
    fig = alt.Chart(data).mark_line().encode(x = "duration", y = "prob").properties(width=500, height=250)
    return fig


In [25]:
fig = plot_ECDF(df)
fig.interactive()

alt.Chart(...)

In [26]:
# fig = plot_ECDF(df)
# fig.interactive()

#### Failure Diagnostics (Aggregate)

Which Vehicles Fail

In [27]:
# Relative failure frequency by vehicle
df = (
    mc_results
    .groupby("anomaly_vehicle")
    .count()
    .rename(columns={'replicant': 'n'})
    .loc[:,'n']
)

fail_count = df.sum()
# (df / fail_count).reset_index()

alt.Chart(
    (df / fail_count).reset_index()
).mark_bar().encode(
    y = "anomaly_vehicle",
    x = "n"
)

alt.Chart(...)

Which Activities Fail?

In [28]:
# Relative failure frequency by activity
df = (
    mc_results
    .groupby("anomaly_activity")
    .count()
    .rename(columns={'replicant': 'n'})
    .loc[:,'n']
)

fail_count = df.sum()
# (df / fail_count).reset_index()

alt.Chart(
    (df / fail_count).reset_index()
).mark_bar().encode(
    y = "anomaly_activity",
    x = "n"
)

alt.Chart(...)

#### Failure Diagnostics (per mission)

Avg ascent failures per mission

In [29]:
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,0,True,72.127086,2,6.000000,Tanker,Countdown
1,0,True,72.127086,3,37.762713,Tanker,Countdown
2,2,True,73.776831,2,6.000000,Tanker,Countdown
3,2,True,73.776831,3,37.762713,Tanker,Countdown
4,2,True,73.776831,4,45.110132,Tanker,Countdown


In [30]:
mc_results.query("anomaly_count > 1")
# df = mc_results.query("replicant == 3")
# df

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,0,True,72.127086,2,6.000000,Tanker,Countdown
1,0,True,72.127086,3,37.762713,Tanker,Countdown
2,2,True,73.776831,2,6.000000,Tanker,Countdown
3,2,True,73.776831,3,37.762713,Tanker,Countdown
4,2,True,73.776831,4,45.110132,Tanker,Countdown
...,...,...,...,...,...,...,...
3213,998,True,76.209632,2,19.000000,Tanker,Countdown
3214,998,True,76.209632,3,37.762713,Tanker,Countdown
3215,998,True,76.209632,4,45.110132,Tanker,Countdown
3216,998,True,76.209632,5,46.757138,Tanker,Countdown


In [31]:
# df.anomaly_activity.value_counts()

In [32]:
df = mc_results.groupby("replicant", as_index=False)["anomaly_activity"].value_counts()
df

,replicant,anomaly_activity,count
0,0,Countdown,2
1,1,Countdown,7
2,2,Countdown,3
3,3,Countdown,3
4,4,Countdown,3
...,...,...,...
1115,995,Orbit Insertion,1
1116,996,Countdown,3
1117,997,Countdown,6
1118,998,Countdown,5


In [33]:
# alt.Chart(df.unstack().Ascent.reset_index()).mark_boxplot().encode(x = "Ascent")

In [34]:
df = (
    mc_results
    # mc_results[mc_results.anomaly_activity != "Countdown"]
    .groupby("replicant", as_index=False)
    ["anomaly_activity"]
    .value_counts()
)

alt.Chart(df).mark_boxplot().encode(
    x = "count",
    y = "anomaly_activity"

)

alt.Chart(...)

In [35]:
df[df.anomaly_activity != "Countdown"]

,replicant,anomaly_activity,count
8,7,Orbit Insertion,1
14,13,Return to Base,1
15,13,Orbit Insertion,1
16,14,Ascent,1
18,15,Orbit Insertion,1
...,...,...,...
1105,988,Ascent,1
1110,992,Orbit Insertion,1
1112,994,Ascent,1
1113,995,Return to Base,1


In [36]:
mc_results

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,0,True,72.127086,2,6.000000,Tanker,Countdown
1,0,True,72.127086,3,37.762713,Tanker,Countdown
2,2,True,73.776831,2,6.000000,Tanker,Countdown
3,2,True,73.776831,3,37.762713,Tanker,Countdown
4,2,True,73.776831,4,45.110132,Tanker,Countdown
...,...,...,...,...,...,...,...
3213,998,True,76.209632,2,19.000000,Tanker,Countdown
3214,998,True,76.209632,3,37.762713,Tanker,Countdown
3215,998,True,76.209632,4,45.110132,Tanker,Countdown
3216,998,True,76.209632,5,46.757138,Tanker,Countdown


---
### Experiments

In [17]:
import pandas as pd
import altair as alt
import numpy as np

# from: https://stackoverflow.com/questions/33345780/empirical-cdf-in-python-similiar-to-matlabs-one
def ecdf(sample):

    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob

sample = [1, 2, 2, 3, 3, 3, 4, 4, 5, 5,]
qe, pe = ecdf(sample)
data = pd.DataFrame({"duration": list(qe), "prob": list(pe)})

alt.Chart(data).mark_line(interpolate='step-after', point=True, color="red").encode(
    x = alt.X("duration", axis=alt.Axis(values=[0, 1, 2, 3, 4, 5]), scale=alt.Scale(domain=(1,5))),
    y = "prob"
).properties(
    width=400, 
    height=400
)

alt.Chart(...)